In [ ]:
from google.colab import drive
drive._mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install catboost

     |████████████████████████████████| 76.3 MB 53 kB/s 


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.model_selection import train_test_split

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Tasks/11/train_data.csv')
test = pd.read_csv('/content/drive/MyDrive/Tasks/11/test_data.csv')

In [ ]:
train.head()

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,Brisbane,13.7,23.6,0.0,5.0,9.6,SSE,24.0,SSW,ESE,7.0,9.0,65.0,53.0,1028.4,1026.7,1.0,1.0,18.3,22.3,0,0
1,Sydney,15.4,21.2,61.2,5.8,2.7,S,43.0,WSW,SSW,19.0,24.0,91.0,65.0,1018.8,1016.5,8.0,7.0,15.6,20.7,1,1
2,Richmond,16.8,30.9,1.0,NaN,NaN,NE,35.0,NNW,NE,7.0,26.0,80.0,38.0,1020.4,1013.2,1.0,NaN,19.6,30.0,0,0
3,NorfolkIsland,13.9,17.1,0.2,3.8,8.9,SE,37.0,SE,SSE,17.0,20.0,56.0,63.0,1027.7,1026.1,3.0,1.0,15.3,15.8,0,0
4,Dartmoor,7.4,14.4,1.8,2.4,8.3,SSW,37.0,NaN,SW,0.0,17.0,100.0,58.0,1026.0,1026.1,NaN,NaN,9.1,13.5,1,0


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135414 entries, 0 to 135413
Data columns (total 22 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Location       135414 non-null  object 
 1   MinTemp        134061 non-null  float64
 2   MaxTemp        134260 non-null  float64
 3   Rainfall       132401 non-null  float64
 4   Evaporation    79262 non-null   float64
 5   Sunshine       73028 non-null   float64
 6   WindGustDir    125722 non-null  object 
 7   WindGustSpeed  125785 non-null  float64
 8   WindDir9am     125506 non-null  object 
 9   WindDir3pm     131665 non-null  object 
 10  WindSpeed9am   133719 non-null  float64
 11  WindSpeed3pm   132786 non-null  float64
 12  Humidity9am    132946 non-null  float64
 13  Humidity3pm    131631 non-null  float64
 14  Pressure9am    121397 non-null  float64
 15  Pressure3pm    121431 non-null  float64
 16  Cloud9am       84221 non-null   float64
 17  Cloud3pm       81547 non-null

In [ ]:
train.Location.unique()

array(['Brisbane', 'Sydney', 'Richmond', 'NorfolkIsland', 'Dartmoor',
       'Mildura', 'Adelaide', 'GoldCoast', 'Woomera', 'Witchcliffe',
       'Uluru', 'Albury', 'Cairns', 'Bendigo', 'Newcastle', 'Moree',
       'Canberra', 'Williamtown', 'PerthAirport', 'MountGambier',
       'Katherine', 'Nhil', 'Perth', 'Melbourne', 'Townsville',
       'WaggaWagga', 'Launceston', 'MountGinini', 'Darwin', 'Ballarat',
       'Watsonia', 'Penrith', 'NorahHead', 'Portland', 'SalmonGums',
       'BadgerysCreek', 'MelbourneAirport', 'Walpole', 'Hobart',
       'CoffsHarbour', 'SydneyAirport', 'Albany', 'Tuggeranong',
       'Nuriootpa', 'AliceSprings', 'Wollongong', 'PearceRAAF', 'Cobar',
       'Sale'], dtype=object)

In [ ]:
#NaN's in categorical
train['WindGustDir'].fillna(method='ffill', inplace=True)
train['WindDir9am'].fillna(method='ffill', inplace=True)
train['WindDir3pm'].fillna(method='ffill', inplace=True)

test['WindGustDir'].fillna(method='ffill', inplace=True)
test['WindDir9am'].fillna(method='ffill', inplace=True)
test['WindDir3pm'].fillna(method='ffill', inplace=True)

In [ ]:
from catboost import CatBoostClassifier, Pool, cv

In [ ]:
cat_columns = ['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm']

In [ ]:
model = CatBoostClassifier(iterations=2500,
                           learning_rate=0.01,
                           depth=8,
                           eval_metric= "F1",
                           task_type="GPU",
                          devices='0:1',
                          silent=True)

In [ ]:
train_pool = Pool(train.drop(columns=['RainTomorrow']), train['RainTomorrow'], cat_features=cat_columns)

In [ ]:
model.fit(train_pool,verbose=False)

In [ ]:
# preds_proba = model.predict_proba(test)[:, 1]
# predict = pd.DataFrame(preds_proba)
preds = model.predict(test)
predict = pd.DataFrame(preds)
predict.to_csv('submit_task11.csv', index=False, header=None)